In [2]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [3]:
# 학습용 데이터 불러오기
path = 'train.csv'
data1 = pd.read_csv(path)

In [4]:
# 평가용 데이터 불러오기
path2 = 'test.csv'
data2 = pd.read_csv(path2)

In [5]:
# 학습용 데이터 확인
data1.head(1)

,id,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,snowing,cloudiness,target
0,TRAIN_00000,27.8816,1013.6,46.8,2.4,232.4,0.0,False,0.0,1.18


In [6]:
data1.describe().T

,count,mean,std,min,25%,50%,75%,max
temperature,19275.0,17.288476,7.192604,-0.208137,11.712,16.530,22.5590,37.058
pressure,19275.0,1059.825712,2544.132430,812.600000,1012.600,1016.800,1021.0000,201373.000
humidity,19275.0,68.948763,14.856929,22.600000,57.800,70.800,80.8000,100.000
wind_speed,19275.0,2.476970,1.359942,0.000000,1.400,2.200,3.2000,29.800
wind_direction,19275.0,166.578179,57.313099,0.000000,127.000,166.600,208.0000,338.000
precipitation,19275.0,0.076572,0.193110,0.000000,0.000,0.000,0.0600,3.180
cloudiness,19275.0,25.749748,17.628612,0.000000,12.000,23.000,37.0000,93.600
target,19275.0,5.477028,3.217581,0.000000,2.956,4.837,7.4365,17.344


In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19275 entries, 0 to 19274
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              19275 non-null  object 
 1   temperature     19275 non-null  float64
 2   pressure        19275 non-null  float64
 3   humidity        19275 non-null  float64
 4   wind_speed      19275 non-null  float64
 5   wind_direction  19275 non-null  float64
 6   precipitation   19275 non-null  float64
 7   snowing         19275 non-null  bool   
 8   cloudiness      19275 non-null  float64
 9   target          19275 non-null  float64
dtypes: bool(1), float64(8), object(1)
memory usage: 1.3+ MB


In [8]:
# 평가용 데이터 확인
data2.head(1)

,id,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,snowing,cloudiness
0,TEST_00000,20.702,1021.0,76.4,2.0,104.8,0.0,False,23.0


# 1. 데이터 전처리

In [9]:
data1 = data1.drop('id',axis=1)
data1.head(1)

,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,snowing,cloudiness,target
0,27.8816,1013.6,46.8,2.4,232.4,0.0,False,0.0,1.18


In [10]:
# 가변수화
dumm_cols = ['snowing']
data1 = pd.get_dummies(data1,columns=dumm_cols,drop_first=True)
data1.head(1)

,temperature,pressure,humidity,wind_speed,wind_direction,precipitation,cloudiness,target,snowing_True
0,27.8816,1013.6,46.8,2.4,232.4,0.0,0.0,1.18,0


In [11]:
# x,y 분리
target = 'target'
x = data1.drop(target,axis=1)
y = data1.loc[:,target]

In [12]:
# 학습용, 평가용 데이터 분리
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.1,random_state=1)

In [13]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_val_s = scaler.transform(x_val)

# 2. 성능 예측

In [14]:
# 불러오기
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score,RandomizedSearchCV,GridSearchCV

from sklearn.metrics import *

In [15]:
# 1. LinearRegression()
model_dt = LinearRegression()
cv_score = cross_val_score(model_dt,x_train,y_train,cv=10)
print(cv_score)
result={}
result['LinearRegression'] = cv_score.mean()

[0.14222208 0.13779577 0.13983364 0.13623216 0.13464562 0.14622926
 0.13568656 0.15428569 0.16909949 0.1526919 ]


In [16]:
# 2. KNN
model_dt = KNeighborsRegressor(n_neighbors=5)
cv_score = cross_val_score(model_dt,x_train,y_train,cv=10)
print(cv_score)
result['KNN'] = cv_score.mean()

[0.09832041 0.05522572 0.07446084 0.03023124 0.07280245 0.08220112
 0.03347823 0.06644799 0.0587119  0.07749054]


In [17]:
# 3. Decision Tree
model_dt = DecisionTreeRegressor(max_depth=5,random_state=1)
cv_score = cross_val_score(model_dt,x_train,y_train,cv=10)
print(cv_score)
result['DecisionTree'] = cv_score.mean()

[0.15170974 0.1590762  0.12618952 0.14820625 0.1317819  0.15434592
 0.15707744 0.14515467 0.16369245 0.1562903 ]


In [ ]:
# 4. SVM
model_dt = SVR(kernel='linear',C=5)
cv_score = cross_val_score(model_dt,x_train,y_train,cv=10)
print(cv_score)
result['SVM'] = cv_score.mean()

In [ ]:
model_dt = 
cv_score = cross_val_score(model_dt,x_train,y_train,cv=10)
print(cv_score)
result[''] = cv_score.mean()

In [ ]:
model_dt = 
cv_score = cross_val_score(model_dt,x_train,y_train,cv=10)
print(cv_score)
result[''] = cv_score.mean()

In [ ]:
model_dt = 
cv_score = cross_val_score(model_dt,x_train,y_train,cv=10)
print(cv_score)
result[''] = cv_score.mean()